In [11]:
from sklearn.datasets import load_iris
from scipy.special import log_softmax
import math
import numpy
from dataclasses import dataclass
import random

from funcs import log_density


In [2]:
load_iris().keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [3]:
load_iris()["data"][0]

array([5.1, 3.5, 1.4, 0.2])

In [4]:
# Create prior weights mean
prior_weights_mean: "numpy.ndarray" = numpy.zeros(shape=(3,4))

# Weights precision
weights_precision: "float" = 1.0

In [65]:
random.choices([1,2],[4,7])[0]

1